In [1]:
import numpy as np
import pysindy as ps
import timeit
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error

In [2]:
dt = loadmat('lorenz_data.mat')['delta_t'][0][0]
x = loadmat('lorenz_data.mat')['tilde_u'][1:30,:,np.newaxis].transpose(1,0,2)
t = dt*np.arange(x.shape[1])+dt
shape = np.concatenate([x.shape[:-1],[4]])
u = np.zeros(shape)
u[:,:,0] = np.cos(t)
u[:,:,1] = (np.roll(x,-1,axis=0))[:,:,0]#u_{j+1}
u[:,:,2] = (np.roll(x,2,axis=0))[:,:,0]
u[:,:,3] = (np.roll(x,1,axis=0))[:,:,0]

## SINDYc

In [3]:
true_coefs = np.array([8,-1,-1,1,-1,1])
true_c = np.zeros([56,])
true_c[0] = true_coefs[0]
true_c[1] = true_coefs[1]
true_c[17] = true_coefs[2]
true_c[19] = true_coefs[3]
true_c[42] = true_coefs[4]
true_c[44] = true_coefs[5]

In [4]:
start = timeit.default_timer()
lib = ps.PolynomialLibrary(degree=3)

In [5]:
opt = ps.STLSQ(alpha=1e-9,threshold=0.8)
model = ps.SINDy(feature_library=lib,optimizer=opt)
model.fit(x, t=dt, u=u)
stop = timeit.default_timer()
model.print(precision=5)
print("Runtime: ", stop-start)
print("Relative coefficient errors: ",((model.optimizer.coef_[0][[0,1,17,19,42,44]]-true_coefs)**2/true_coefs**2)**0.5)
print("Total error: ",np.linalg.norm((model.optimizer.coef_[0]-true_c), ord=None, axis=None, keepdims=False)/np.linalg.norm(true_c))

(x0)' = 8.00000 1 + -0.99999 x0 + -0.99189 u1 u3 + 0.98105 u2 u3 + -1.00809 u0 u1 u3 + 1.01895 u0 u2 u3
Runtime:  0.12611979991197586
Relative coefficient errors:  [8.38552479e-08 5.15763061e-06 8.10549916e-03 1.89470231e-02
 8.09466572e-03 1.89469698e-02]
Total error:  0.0035081687351230673


#### Weak SINDy

In [6]:
true_coefs = np.array([8,-1,-1,1,-1,1])
true_c = np.zeros([26,])
true_c[0] = true_coefs[0]
true_c[1] = true_coefs[1]
true_c[14] = true_coefs[2]
true_c[15] = true_coefs[3]
true_c[23] = true_coefs[4]
true_c[24] = true_coefs[5]

In [7]:
start = timeit.default_timer()
grid_shape = np.concatenate([x.shape[:-1],[2]])
spatiotemporal_grid = np.zeros(grid_shape)
spatiotemporal_grid[:,:,0] = np.arange(128)[:,np.newaxis]
spatiotemporal_grid[:,:,1] = t
np.random.seed(100)

In [8]:
weak_lib = ps.WeakPDELibrary(
    library_functions = [lambda x:x, lambda x,y: x*y, lambda x,y,z: x*y*z],
    function_names = [lambda x:x, lambda x,y:x+y,lambda x,y,z:x+y+z],
    derivative_order = 0,
    spatiotemporal_grid = spatiotemporal_grid,
    K = 5000,
    H_xt = [10,0.2/10],
    include_bias = True)

In [9]:
opt = ps.STLSQ(alpha=1e-15,threshold=0.5)
model = ps.SINDy(feature_library=weak_lib,optimizer=opt)
model.fit(x, t = dt, u = u)
stop = timeit.default_timer()
model.print(precision = 5)

print("Weakform Runtime: ", stop-start)
print("Weakform Relative coefficient errors:",((model.optimizer.coef_[0][[0,1,14,15,23,24]]-true_coefs)**2/true_coefs**2)**0.5)
print("Total error: ",np.linalg.norm((model.optimizer.coef_[0]-true_c), ord=None, axis=None, keepdims=False)/np.linalg.norm(true_c))

(x0)' = 8.00001 1 + -0.99999 x0 + -0.99725 u1u3 + 0.99496 u2u3 + -1.00276 u0u1u3 + 1.00504 u0u2u3
Weakform Runtime:  5.422908899956383
Weakform Relative coefficient errors: [9.05566570e-07 8.19383375e-06 2.75115341e-03 5.04087289e-03
 2.75994427e-03 5.04060949e-03]
Total error:  0.000978051898912502
